In [58]:
import os
import numpy as np
from keras.models import Model
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Dense, GlobalAveragePooling2D
from PIL import Image, ImageChops, ImageEnhance
import cv2
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from sklearn.model_selection import train_test_split
from scipy import stats
from keras.models import Sequential
from keras.layers import BatchNormalization
from tensorflow.keras.applications import Xception
from sklearn.metrics import classification_report

In [2]:
def convert_to_ela_image(path, quality):
    """
    Performs Error Level Analysis (ELA) on an image.
    path (str) - the path to the image file.
    quality (int) - quality to which reducing the image

    Returns:
        elaImage (np.array) - the ELA image.
    """
    
    temp_filename = 'temp.jpg'
    image = Image.open(path).convert('RGB')
    image.save(temp_filename, 'JPEG', quality = quality)
    temp_image = Image.open(temp_filename)
    ela_image = ImageChops.difference(image, temp_image)
    extrema = ela_image.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    scale = 255.0 / max_diff
    ela_image = ImageEnhance.Brightness(ela_image).enhance(scale)
    return ela_image

In [15]:
def build_xception_model(num_classes=2):
    base_model = Xception(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

    for layer in base_model.layers:
        layer.trainable = False

    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    return model

In [4]:
def prepare_image(image_path):
    return np.array(convert_to_ela_image(image_path, 90).resize(image_size)).flatten() / 255.0

In [48]:
image_size = (299, 299)
X = []
y = []

In [6]:
# Define source paths
authentic = 'data/casia/au/'
tampered = 'data/casia/tp/'

In [49]:
for root, dirs, files in os.walk(authentic):
  for filename in files:
    file_path = os.path.join(root, filename)
    if filename.lower().endswith(('jpg', 'png')):
      X.append(prepare_image(file_path))
      y.append(1)
    if len(X)==3000:
        break

# random.shuffle(X)

print(len(X), len(y))

37 37


In [50]:
count = 0
for root, dirs, files in os.walk(tampered):
    for filename in files:
        file_path = os.path.join(root, filename)
        if filename.lower().endswith(('jpg', 'png', 'tif', 'tiff')):
            X.append(prepare_image(file_path))
            y.append(0)
        if len(X)==4500:
            break

In [51]:
X = np.array(X)
y = to_categorical(y, 2)
y = y.reshape(y.shape[0], 2)
X = X.reshape(-1, 299, 299, 3)

In [53]:
X_train, X_val, Y_train, Y_val = train_test_split(X, y, test_size = 0.2, random_state=5)
X = X.reshape(-1,1,1,1)
print(len(X_train), len(Y_train))
print(len(X_val), len(Y_val))

58 58
15 15


In [12]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5,mode='max', restore_best_weights=True)
early_stopping

In [54]:
model = build_xception_model()
epochs = 30
batch_size = 32
model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 10, 10, 2048)      20861480  
                                                                 
 flatten_1 (Flatten)         (None, 204800)            0         
                                                                 
 batch_normalization_14 (Bat  (None, 204800)           819200    
 chNormalization)                                                
                                                                 
 dense_4 (Dense)             (None, 256)               52429056  
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 2)                 514       
                                                      

In [55]:
hist = model.fit(
    X_train,
    Y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_data = (X_val, Y_val),
    callbacks = [early_stopping])
hist

Epoch 1/30
2/2 [==============================] - 5s 2s/step - loss: 3.6082 - accuracy: 0.5862 - val_loss: 2.7422 - val_accuracy: 0.9333
Epoch 2/30
2/2 [==============================] - 3s 1s/step - loss: 2.6168 - accuracy: 0.9655 - val_loss: 3.0525 - val_accuracy: 0.9333
Epoch 3/30
2/2 [==============================] - 3s 1s/step - loss: 1.4188 - accuracy: 0.9655 - val_loss: 3.2747 - val_accuracy: 0.9333
Epoch 4/30
2/2 [==============================] - 3s 1s/step - loss: 1.0565 - accuracy: 1.0000 - val_loss: 3.1538 - val_accuracy: 0.9333
Epoch 5/30
2/2 [==============================] - 3s 1s/step - loss: 2.7425e-06 - accuracy: 1.0000 - val_loss: 3.0667 - val_accuracy: 0.9333
Epoch 6/30
2/2 [==============================] - 3s 1s/step - loss: 0.7412 - accuracy: 0.9828 - val_loss: 2.9368 - val_accuracy: 0.9333


In [56]:
model.evaluate(X_val,Y_val)

1/1 [==============================] - 0s 499ms/step - loss: 2.7422 - accuracy: 0.9333


[2.742173194885254, 0.9333333373069763]

In [59]:
y_test_labels = np.argmax(Y_val, axis=1)
y_pred_probs = model.predict(X_val)
y_pred_labels = np.argmax(y_pred_probs, axis=1)
print(classification_report(y_test_labels, y_pred_labels, target_names=['Class 0', 'Class 1']))

1/1 [==============================] - 1s 510ms/step
              precision    recall  f1-score   support

     Class 0       0.90      1.00      0.95         9
     Class 1       1.00      0.83      0.91         6

    accuracy                           0.93        15
   macro avg       0.95      0.92      0.93        15
weighted avg       0.94      0.93      0.93        15

